In [1]:
import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()



gpt_4o = ChatOpenAI(model="gpt-4o", temperature=0)
gpt_4o_mini = ChatOpenAI(model="gpt-4o-mini", temperature=0)

api_key = os.environ["OPENAI_API_KEY"]

### PROCESAMOS LA DATA CON PANDAS

In [23]:
import pandas as pd

#Ponemos la ruta
path = "D:\Cursos\Curso-BOOTCAMP-IA-GENERATIVA-SOLUCIONES-CON-PYTHON\TMDB_movie_dataset_v11.csv"
#Leemos la ruta
df = pd.read_csv(path)

#Convertimos los generos a minúsculas
df['genres'] = df['genres'].str.lower()

#Filtramos
filtered_df = df[
    df['genres'].str.contains('action') &
    df['genres'].str.contains('fiction')
]

#Seleccionamos solo 1000 registros
df_1000 = filtered_df.sample(n=1000)

print(df_1000)

             id                                              title  \
890915   809463                           Immortal Crime: Part One   
129047   640394                     Dragon Force: So Long Ultraman   
71747     30790                                    Six Reasons Why   
11419    327029                                      Power/Rangers   
539096  1239422                        The Lost World: Underground   
...         ...                                                ...   
732077   425832                                            TauTona   
76550    625985          Kamen Rider Zi-O NEXT TIME: Geiz, Majesty   
62748     53502                                    The Dead Undead   
34122     97262  Bionic Showdown: The Six Million Dollar Man an...   
298609   961071  Kamen Rider Genms: -Smart Brain and the 1000% ...   

        vote_average  vote_count    status release_date  revenue  runtime  \
890915         0.000           0  Released   2016-03-28        0        8   
12904

In [13]:
df_1000['data'] = df_1000.apply(
    lambda row: f"Titulo: {row['title']}, Estado: {row['status']}, Sinopsis: {row['overview']}, "
                f"Generos: {row['genres']}, Empresas_productoras: {row['production_companies']}, "
                f"Idiomas_hablados: {row['spoken_languages']}, palabras_clave: {row['keywords']}",
    axis=1
)
print(df_1000['data'])

126        Titulo: World War Z, Estado: Released, Sinopsi...
498        Titulo: The Meg, Estado: Released, Sinopsis: A...
676        Titulo: Resident Evil, Estado: Released, Sinop...
826        Titulo: Alien³, Estado: Released, Sinopsis: Af...
956        Titulo: Alien Resurrection, Estado: Released, ...
                                 ...                        
1039082    Titulo: Aurora, Estado: In Production, Sinopsi...
1046955    Titulo: Cabal, Estado: Released, Sinopsis: Tra...
1055916    Titulo: Anomaly, Estado: In Production, Sinops...
1058936    Titulo: Kamen Rider Ghost: Final Stage, Estado...
1067197    Titulo: Другое измерение, Estado: Released, Si...
Name: data, Length: 632, dtype: object


C:\Users\Carlos\AppData\Local\Temp\ipykernel_24860\587645697.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1000['data'] = df_1000.apply(


In [15]:
ruta_salida = 'D:\Cursos\Curso-BOOTCAMP-IA-GENERATIVA-SOLUCIONES-CON-PYTHON/movies_dataset_oficial.csv'
df_1000.to_csv(ruta_salida, index=False)

## Load VectorDB

In [4]:
import pandas as pd

df = pd.read_csv('D:\Cursos\Curso-BOOTCAMP-IA-GENERATIVA-SOLUCIONES-CON-PYTHON/movies_dataset_oficial.csv')
# df = pd.read_csv('./movies_dataset_oficial.csv')

ids = df['id'].astype(str).tolist()
documents = df['data'].tolist()
metadatas = []

for adult, release_date in zip(df['adult'].tolist(), df['release_date'].tolist()):
    metadata = {
        'adult': adult,
        'release_date': release_date
    }
    metadatas.append(metadata)


In [38]:
print(f"ids: Type={type(ids[0])} Sample={ids[:5]}")
print(f"documents: Type={type(documents[0])} Sample={documents[:5]}")
print(f"metadatas: Type={type(metadatas[0])} Sample={metadatas[:5]}")

ids: Type=<class 'str'> Sample=['72190', '345940', '1576', '8077', '8078']
documents: Type=<class 'str'> Sample=['Titulo: World War Z, Estado: Released, Sinopsis: Life for former United Nations investigator Gerry Lane and his family seems content. Suddenly, the world is plagued by a mysterious infection turning whole human populations into rampaging mindless zombies. After barely escaping the chaos, Lane is persuaded to go on a mission to investigate this disease. What follows is a perilous trek around the world where Lane must brave horrific dangers and long odds to find answers before human civilization falls., Generos: action, drama, horror, science fiction, thriller, Empresas_productoras: GK Films, Paramount, Hemisphere Media Capital, 2DUX², Apparatus Productions, Latina Pictures, Skydance, Plan B Entertainment, Idiomas_hablados: English, palabras_clave: philadelphia, pennsylvania, new jersey, based on novel or book, dystopia, jerusalem, israel, apocalypse, zombie, epidemic, nuclea

### Base de datos Vectorial

In [5]:
import chromadb
from chromadb.utils import embedding_functions

chroma_client = chromadb.Client()

In [6]:
api_key = os.environ["OPENAI_API_KEY"]
collection = chroma_client.create_collection(
    name="movies_collection",
    embedding_function=embedding_functions.OpenAIEmbeddingFunction(
        api_key=api_key,
        model_name="text-embedding-3-small"
        )
    )

In [9]:
chroma_client.delete_collection(name="movies_collection")

In [1]:
def calculate_indices(batch_index, batch_size, total_items):
    start_index = batch_index * batch_size
    end_index = min((batch_index + 1) * batch_size, total_items)
    return start_index, end_index


batch_size = 100

total_items = len(ids)
num_batches = (total_items + batch_size - 1) // batch_size

# Iterate over the batches
for batch_index in range(num_batches):
    start_index, end_index = calculate_indices(batch_index, batch_size, total_items)
    response = collection.add(
        ids=ids[start_index:end_index],
        documents=documents[start_index:end_index],
        metadatas=metadatas[start_index:end_index]
    )

NameError: name 'ids' is not defined

In [27]:
results = collection.query(
    query_texts=["battle", "mission"],
    n_results=1,
    where={"adult": False or True}
)

print(results)



In [31]:
results['ids']

[['11926', '1062839', '936252']]

### TOOLS

In [2]:
import requests
from dotenv import load_dotenv
import os


load_dotenv()
API_KEY_TMDB = os.getenv('API_KEY_TMDB')
URL_TMBD = 'https://api.themoviedb.org/4/movie/recommendations'



def search_movie(page: int):
    # Parámetros
    params = {
        'page': page,
        'language': 'en-US'
    }  

    #Colocamos el código de autorización
    headers = {
        'Authorization': f'Bearer {API_KEY_TMDB}',
        'accept': 'application/json'
    }

    response = requests.get(URL_TMBD, headers=headers, params=params)

    if response.status_code == 200:
        return response.json()
    else:
        return {'Error': f"{response.status_code}"}
    
peliculas= search_movie(1)
print(peliculas)



{'Error': '404'}


In [3]:
import requests
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY_TMDB = os.getenv('API_KEY_TMDB')
account = os.getenv('account')
BASE_URL_TMBD_V4 = f"https://api.themoviedb.org/4/"


def get_headers():
    """Genera los encabezados necesarios para la petición a la API"""
    return {
        "accept": "application/json",
        'Authorization': f'Bearer {API_KEY_TMDB}'
    }



def handle_response(response):
    """Maneja la respuesta de la API, devolviendo un JSON si es exitosa o un error si falla"""

    if response.status_code == 200:
        return response.json()
    else:
        return {'Error': f"{response.status_code}: {response.reason}"}

#Funcion para buscar películas por título
#Función para buscar series recomendadas por la api
def recommendedMovies(page: int):
    """Obtiene recomendaciones de películas"""
    endpoint = f'account/{account}/tv/recommendations'
    url = f"{BASE_URL_TMBD_V4}{endpoint}"

    # Parámetros
    params = {
        'page': page,
        'language': 'en-US'
    }  

    try:
        response = requests.get(url, headers=get_headers(), params=params)
        return handle_response(response)
    except requests.exceptions.RequestException as e:
        return {'Error': f"Request failed: {e}"}
    

peliculas = recommendedMovies(2)
print(peliculas)

{'page': 2, 'results': [{'backdrop_path': '/iVDDs99Itou2e2xnmxwiNjtlzYC.jpg', 'id': 18165, 'name': 'The Vampire Diaries', 'original_name': 'The Vampire Diaries', 'overview': 'The story of two vampire brothers obsessed with the same girl, who bears a striking resemblance to the beautiful but ruthless vampire they knew and loved in 1864.', 'poster_path': '/aBkVgChtyyJaHyZh1gfd8DbzQon.jpg', 'media_type': 'tv', 'adult': False, 'original_language': 'en', 'genre_ids': [18, 10765], 'popularity': 572.734, 'first_air_date': '2009-09-10', 'vote_average': 8.333, 'vote_count': 8884, 'origin_country': ['US']}, {'backdrop_path': '/1i1N0AVRb54H6ZFPDTwbo9MLxSF.jpg', 'id': 85271, 'name': 'WandaVision', 'original_name': 'WandaVision', 'overview': 'Wanda Maximoff and Vision—two super-powered beings living idealized suburban lives—begin to suspect that everything is not as it seems.', 'poster_path': '/glKDfE6btIRcVB5zrjspRIs4r52.jpg', 'media_type': 'tv', 'adult': False, 'original_language': 'en', 'genre_i

In [4]:
import requests
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY_TMDB = os.getenv('API_KEY_TMDB')
BASE_URL_TMBD = 'https://api.themoviedb.org/3/'


def get_headers():
    """Genera los encabezados necesarios para la petición a la API"""
    return {
        "accept": "application/json",
        'Authorization': f'Bearer {API_KEY_TMDB}',
    }


def handle_response(response):
    """Maneja la respuesta de la API, devolviendo un JSON si es exitosa o un error si falla"""

    if response.status_code == 200:
        return response.json()
    else:
        return {'Error': f"{response.status_code}: {response.reason}"}


#PETICIONES 


#Funcion para buscar películas que estén en los cines actualmente 
def  movie_nowPlaying(page: int):
    """Busca películas por título usando la API de TMDB"""

    endpoint = 'movie/now_playing'
    url = f"{BASE_URL_TMBD}{endpoint}"

    # Parámetros
    params = {
        'language': 'en-US',
        'page': page
    }  

    try:
        response = requests.get(url, headers=get_headers(), params=params)
        return handle_response(response)
    except requests.exceptions.RequestException as e:
        return {'Error': f"Request failed: {e}"}
    

def top_rated(page: int):
    """Obtiene las películas mejor valoradas de la historia en orden decreciente"""

    endpoint = 'movie/top_rated'
    url = f"{BASE_URL_TMBD}{endpoint}"

    # Parámetros
    params = {
        'language': 'en-US',
        'page': page
    }  

    try:
        response = requests.get(url, headers=get_headers(), params=params)
        return handle_response(response)
    except requests.exceptions.RequestException as e:
        return {'Error': f"Request failed: {e}"}
    

def top_rated_TV(page: int):
    """Obtiene las series mejor valoradas de la historia en orden decreciente"""

    endpoint = 'tv/top_rated'
    url = f"{BASE_URL_TMBD}{endpoint}"

    # Parámetros
    params = {
        'language': 'en-US',
        'page': page
    }  

    try:
        response = requests.get(url, headers=get_headers(), params=params)
        return handle_response(response)
    except requests.exceptions.RequestException as e:
        return {'Error': f"Request failed: {e}"}
        

print(top_rated(1))


{'page': 1, 'results': [{'adult': False, 'backdrop_path': '/zfbjgQE1uSd9wiPTX4VzsLi0rGG.jpg', 'genre_ids': [18, 80], 'id': 278, 'original_language': 'en', 'original_title': 'The Shawshank Redemption', 'overview': 'Imprisoned in the 1940s for the double murder of his wife and her lover, upstanding banker Andy Dufresne begins a new life at the Shawshank prison, where he puts his accounting skills to work for an amoral warden. During his long stretch in prison, Dufresne comes to be admired by the other inmates -- including an older prisoner named Red -- for his integrity and unquenchable sense of hope.', 'popularity': 160.678, 'poster_path': '/9cqNxx0GxF0bflZmeSMuL5tnGzr.jpg', 'release_date': '1994-09-23', 'title': 'The Shawshank Redemption', 'video': False, 'vote_average': 8.706, 'vote_count': 26780}, {'adult': False, 'backdrop_path': '/tmU7GeKVybMWFButWEGl2M4GeiP.jpg', 'genre_ids': [18, 80], 'id': 238, 'original_language': 'en', 'original_title': 'The Godfather', 'overview': 'Spanning t

In [5]:
from langchain_core.tools import tool
from langchain.schema import SystemMessage, HumanMessage
import json

WRITER_SYSTEM_MOVIE= """
You will receive the API request data with the title of the searched movie as "title" and the API response content as "content". Your task is to:

1. Create a friendly synopsis that describes what the movie is about based on the information provided.
  
2. Extract and return only the relevant information for movies whose title exactly matches the "title" provided by the user. If the content includes movies unrelated to the requested title, ignore them.

3. If you detect a possible spelling error in the title, notify the user and suggest the correct title based on your understanding.

4. If the title is correct, provide the essential information about the matching movie(s) found within the API content. If the movie is part of a franchise and there are multiple related movies, include them as well.

Make sure the friendly synopsis is informative and engaging, but do not invent or add any movies that are not present in the content provided.
"""

@tool
def search_title(title: str):
    """Use this tool when the user requests detailed information about a specific movie. 
    The tool searches for movies with the exact title provided by the user and returns movies containing that title and
    relevant information. If the title is misspelled, it suggests a correction. 
    """
    
    #Esta variable almacenará las películas encontradas con el title
    movies = search_movie(title)

    #Validamos
    if not movies:
        return f"Lo siento, no encontramos películas con ese título '{title}'"
    
    #Mejoramos el formato
    movies_content = json.dumps(movies)

    #mandamos el contenido a un modelo gpt para que lo filtre
    try:
        response = gpt_4o_mini.invoke(
            input=[
                SystemMessage(content=WRITER_SYSTEM_MOVIE),
                HumanMessage(content=f"Title: {title}, content: {movies_content}")

            ]
        ).content
    except Exception as e:
        return f"Ha ocurrido un error procesando tu petición: {str(e)}"
    

    formated_response = f"## Movie Information: {title}\n\n{response}"

    return formated_response

    
@tool
def apiMovieRecommendations(page: int):
    """Use this tool when the user asks for non-personalized recommendations, 
    when he/she only wants you to recommend or list good movies."""

    apiMovies = recommendedMovies(page)

    if not apiMovies:
        return f"Lo siento no tengo recomendaciones rápidas ahora mismo " 
    
    moviesContent = json.dumps(apiMovies)

    formated_response = f"## Recommended Movies: {moviesContent}"
    
    return formated_response


@tool
def TvTopRated(page: int):
    """use this tool to get the best rated series in history with their small sipnopsis. 
    They will be in decreasing order
    
    Arguments:
    page -- the page of results to fetch (pagination)

    """
    #Validación de entrada
    if not isinstance(page, int) or page <= 0:
        return "Error: El número de página debe ser un entero positivo."

    try:
        #Llamada a la función que obtiene datos de la api
        tv_shows = top_rated_TV(page)

        if not tv_shows or 'results' not in tv_shows:
            return "No ha sido posible encontrar las series"

        #Procesar los datos para mostrar la información que yo quiera
        formatted_shows = []
        for show in tv_shows['results']:
            formatted_shows.append({
                'id': show.get('id', 'id no disponible'),
                'title': show.get('name', 'Título desconocido'),
                'genre_ids': show.get('genre_ids', 'Géneros no disponibles'),
                'overview': show.get('overview', 'Sinopsis no disponible'),
                'vote_average': show.get('vote_average', 'rate no disponible'),
                'release_date': show.get('release_date', 'año de lanzamiento no disponible'),
                'poster_path': show.get('poster_path', 'foto no disponible'),

            })

        #Crear la estructura del JSON de respuesta
        response_data = {
            "page": page,
            "total_results": tv_shows.get('total_results', 0),
            "total_pages": tv_shows.get('total_pages', 0),
            "shows": formatted_shows
        }


        response_format = json.dumps(response_data, indent=4)

        return f"## TV SHOWS Top Rated (Page {page}):\n{response_format}"

    except Exception as e:
        #Manejo de errores para problemas inesperados
        return f"Error al obtener las series: {str(e)}"    



tools = [search_title, apiMovieRecommendations, TvTopRated]

In [8]:
import json
movies = search_movie("shrek")
movies_content = json.dumps(movies)

print(movies_content)

{"page": 1, "results": [{"adult": false, "backdrop_path": "/sRvXNDItGlWCqtO3j6wks52FmbD.jpg", "genre_ids": [16, 35, 14, 12, 10751], "id": 808, "original_language": "en", "original_title": "Shrek", "overview": "It ain't easy bein' green -- especially if you're a likable (albeit smelly) ogre named Shrek. On a mission to retrieve a gorgeous princess from the clutches of a fire-breathing dragon, Shrek teams up with an unlikely compatriot -- a wisecracking donkey.", "popularity": 130.708, "poster_path": "/iB64vpL3dIObOtMZgX3RqdVdQDc.jpg", "release_date": "2001-05-18", "title": "Shrek", "video": false, "vote_average": 7.745, "vote_count": 16868}, {"adult": false, "backdrop_path": "/xZ2we4gdiwQmg6D1w9qHlAm5yIf.jpg", "genre_ids": [35, 12, 14, 16, 10751], "id": 10192, "original_language": "en", "original_title": "Shrek Forever After", "overview": "A bored and domesticated Shrek pacts with deal-maker Rumpelstiltskin to get back to feeling like a real ogre again, but when he's duped and sent to a

### AGENTE

In [6]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain.schema import SystemMessage


INSTRUCTION = """
Eres un asistente especializado en recomendaciones de películas y series para amantes del cine. Tu tarea principal es hablar exclusivamente sobre temas relacionados con películas y series, y responder consultas dentro de este ámbito.

- Interacciones Iniciales:
Si el usuario comienza la conversación con una consulta o pidiendo una recomendación, responde de manera natural y directa.
Si el usuario inicia la conversación con un saludo o pregunta sobre tu función, ofrece una breve introducción sobre cómo puedes ayudar y las funcionalidades que tienes disponibles. 

Ejemplo:

¡Hola, cinéfilo!

¿Listo para descubrir tu próxima gran película o serie? Puedo recomendarte algo personalizado
basado en tus gustos, o si prefieres, podemos buscar por filtros o palabras clave. 
¿Qué te apetece hoy? 🎬🍿


Puedes utilizar o realizar variaciones de este ejemplo que te acabo de dar para poder iniciar
la conversación.

- Uso de Herramientas:

Herramienta "search_title":
Cuándo usarla: Si el usuario solicita información específica sobre una película o serie.
Cómo usarla: Envía la consulta del usuario a esta herramienta para devolver la información solicitada. Asegúrate de que la respuesta incluya solo la información relevante y necesaria.

Herramienta "recommendation":
Cuándo usarla: Si el usuario pide una recomendación personalizada.
Proceso de Recomendación:
Haz mínimo 4 preguntas antes de usar la herramienta.
No hagas todas las preguntas de golpe; espera a que el usuario responda cada una antes de formular la siguiente.
Realiza preguntas de manera progresiva para recopilar información importante. Pregunta sobre:
- Descripción de lo que busca.
- Película o serie reciente que le gustó.
- Qué aspectos disfrutó más de su última película o serie.
- Géneros o temas que le interesan.
Cómo usarla: Una vez recopilada la información necesaria, utiliza la herramienta "recommendation" y asegúrate de enviar la información del usuario en inglés. Luego, presenta la recomendación en español y complementa con detalles adicionales de la película o serie recomendada si es necesario.

Herramienta "apiMovieRecommendations":
Cuándo usarla: Cuando el usuario desee una recomendación rápida de películas, las mejores películas del momento, más populares o una recomendación no personalizada.
Cómo usarla: Llama a la herramienta para obtener las mejores películas según la API. Si el usuario pide más películas, vuelve a usar la herramienta con un número de página incrementado (pasando el número 2 para la segunda página, y así sucesivamente), hasta que ya no se devuelvan más resultados.

Herramienta "findMovies_NowPlaying":
Cuándo usarla: Cuando el usuario pida las películas que se encuentren en cine actualmente, en cartelera, cuando el usuario quiera ver una película en el cine, cuando el usuario te pregunte que películas hay en el cine.
Cómo usarla: Llama a la herramienta para obtener las películas que se encuentren en cine o cartelera actualmente, cada película tendrá su pequeña sipnopsis. Si el usuario pide más películas, vuelve a usar la herramienta con un número de página incrementado (pasando el número 2 para la segunda página, y así sucesivamente), hasta que ya no se devuelvan más resultados.

Herramienta "moviesTopRated":
Cuándo usarla: Cuando el usuario pida las películas mejor valoradas en general, las más valoradas del mundo, las más valoradas de la historia o las mejores del mundo.
Cómo usarla: Llama a la herramienta para obtener las películas mejor valoradas de la historia en orden decreciente(de mayor a menor), cada película tendrá su pequeña sipnopsis y su valor de rating. Si el usuario pide más películas, vuelve a usar la herramienta con un número de página incrementado (pasando el número 2 para la segunda página, y así sucesivamente), hasta que ya no se devuelvan más resultados.

Herramienta "apiTVShowsRecommendations":
Cuándo usarla: Cuando el usuario desee una recomendación rápida de series, las mejores series mejor valoradas, más populares o una recomendación no personalizada.
Cómo usarla: Llama a la herramienta para obtener las mejores series según la API. Si el usuario pide más series, vuelve a usar la herramienta con un número de página incrementado (pasando el número 2 para la segunda página, y así sucesivamente), hasta que ya no se devuelvan más resultados.

Herramienta "TvTopRated":
Cuándo usarla: Cuando el usuario pida las series mejor valoradas en general, las más valoradas del mundo, las más valoradas de la historia o las mejores del mundo.
Cómo usarla: Llama a la herramienta para obtener las series mejor valoradas de la historia en orden decreciente(de mayor a menor), cada serie tendrá su pequeña sipnopsis y su valor de rating. Si el usuario pide más series, vuelve a usar la herramienta con un número de página incrementado (pasando el número 2 para la segunda página, y así sucesivamente), hasta que ya no se devuelvan más resultados.

Generos: 
Los géneros que te brinden las herramientas serán en ids, solo te darán ids y tu tienes que saber que género pertenece a cada id. 
Cuando recibas un ID de género, utilízalo para identificar el nombre del género correspondiente. Los géneros y sus respectivos IDs son los siguientes:
28 → Action
12 → Adventure
16 → Animation
35 → Comedy
80 → Crime
99 → Documentary
18 → Drama
10751 → Family
14 → Fantasy
36 → History
27 → Horror
10402 → Music
9648 → Mystery
10749 → Romance
878 → Science Fiction
10770 → TV Movie
53 → Thriller
10752 → War
37 → Western
10759 → Action & Adventure
10762 → Kids
10763 → News
10764 → Reality
10765 → Sci-Fi & Fantasy
10766 → Soap
10767 → Talk
10768 → War & Politics




Idioma:
Interacción: Toda la conversación, exceptuando los títulos de las películas, debe ser en español.
Títulos: Los títulos de las películas o series pueden presentarse en inglés, pero el resto de la respuesta debe estar en español.

"""

memory = MemorySaver()

MovieAgent = create_react_agent(
    gpt_4o,
    tools=tools,
    state_modifier=SystemMessage(content=INSTRUCTION),
    checkpointer=memory
)


### RESPONSE

In [7]:
msg = """
hola, quiero saber cuales son las mejores series
"""

response = MovieAgent.invoke(
    input={"messages": msg},
    config={
        "configurable": {

            "thread_id": 9
        }
    }
)
 

for message in response["messages"]:
   print(f"{message.type.upper()}: {message.content}")
   print("")

HUMAN: 
hola, quiero saber cuales son las mejores series


AI: ¡Hola, cinéfilo!

Voy a buscar las mejores series de todos los tiempos para ti. Dame un momento y te traigo la información. 🎬🍿



TOOL: ## TV SHOWS Top Rated (Page 1):
{
    "page": 1,
    "total_results": 2033,
    "total_pages": 102,
    "shows": [
        {
            "id": 1396,
            "title": "Breaking Bad",
            "genre_ids": [
                18,
                80
            ],
            "overview": "Walter White, a New Mexico chemistry teacher, is diagnosed with Stage III cancer and given a prognosis of only two years left to live. He becomes filled with a sense of fearlessness and an unrelenting desire to secure his family's financial future at any cost as he enters the dangerous world of drugs and crime.",
            "vote_average": 8.914,
            "release_date": "a\u00f1o de lanzamiento no disponible",
            "poster_path": "/ztkUQFLlC19CCMYHW9o1zWhJRNq.jpg"
        },
        {
       

### Transición de CSV

In [ ]:
import csv

#Aquí pondremos el archivo csv ya hecho
with open('') as file:
    lines = csv.reader(file)

    documents = []
    metadatas = []
    ids = []

    for i, line in enumerate(lines):
        if i == 0:
            continue
        
        #Aquí señalaremos que columna le pasaremos a documents
        documents.append(line[26])
        metadatas.append({"adult_content": line[8]})
        ids.append(str(line[0]))



In [17]:
peliculas = [
    {
       #"id": 27205,
       "title": "Inception", 
       "status": "Released", 
       #"runtime": 148,  
       #"adult": False,  
       "overview": "Cobb, a skilled thief who commits corporate espionage by infiltrating the subconscious of his targets is offered a chance to regain his old life as payment for a task considered to be impossible: \"inception\", the implantation of another person's idea into a target's subconscious.",  
       #"tagline": "Your mind is the scene of the crime.",  
       "genres": "Action, Science Fiction, Adventure",  
       "production_companys": "Legendary Pictures, Syncopy, Warner Bros. Pictures",  
       #"production_countries": "United Kingdom, United States of America",  
       "spoken_languages": "English, French, Japanese, Swahili",  
       "keywords": "rescue, mission, dream, airplane, paris, france, virtual reality, kidnapping, philosophy, spy",                                   
    }
]

document = peliculas[0].__str__()
print(document)

{'title': 'Inception', 'status': 'Released', 'overview': 'Cobb, a skilled thief who commits corporate espionage by infiltrating the subconscious of his targets is offered a chance to regain his old life as payment for a task considered to be impossible: "inception", the implantation of another person\'s idea into a target\'s subconscious.', 'genres': 'Action, Science Fiction, Adventure', 'production_companys': 'Legendary Pictures, Syncopy, Warner Bros. Pictures', 'spoken_languages': 'English, French, Japanese, Swahili', 'keywords': 'rescue, mission, dream, airplane, paris, france, virtual reality, kidnapping, philosophy, spy'}


## Probamos Peticiones a postgres

In [8]:
from dotenv import load_dotenv
import os
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

# Carga las variables de entorno
load_dotenv()

user = os.getenv('DB_USER')
password = os.getenv('DB_PASSWORD')
host = os.getenv('DB_HOST')  # 'db' es el nombre del servicio en Docker Compose
port = os.getenv('DB_PORT')  # Puerto por defecto de PostgreSQL
db_name = os.getenv('DB_NAME')

# Formatear la URL de la base de datos
DATABASE_URL = f"postgresql://{user}:{password}@{host}:{port}/{db_name}"

# Crear el engine de SQLAlchemy
engine = create_engine(DATABASE_URL)

# Creamos el factory para las sesiones
SessionLocal = sessionmaker(bind=engine)

def get_db():
    """Crea y devuelve una sesión de base de datos para consultas"""
    db = SessionLocal()
    try:
        yield db
    finally:
        db.close()


In [12]:
from sqlalchemy import text
from sqlalchemy.exc import SQLAlchemyError

def get_oscar_by_actor(name: str):
    """Esta función obtiene los oscars a los que ha sido nominado el actor"""

    # Validación de parámetros
    if not isinstance(name, str) or not name.strip():
        raise ValueError("El nombre del actor debe ser una cadena no vacía.")

    # Obtener la sesión
    try:
        db = next(get_db())
    except StopIteration:
        raise RuntimeError("No se pudo obtener una sesión de base de datos.") 

    try:

        #Realizamos la consulta
        query = f"%{name}%"
        result = db.execute(
            text("SELECT * FROM oscar_awards WHERE actor_name ILIKE :name"),
            {'name': query}
        ).fetchall()

        if not result: 
            return {"message": "No se encontraron premios oscar para el actor especificado"}
        
        return result

    except SQLAlchemyError as e:
        #Manejo de errores específicos de SQLAlchemy
        print(f"Error al ejecutar la consulta: {e}")
        return {"error": "Error en la consulta a la base de datos"}
    except Exception as e:
        #Manejo de errores generales
        print(f"Se produjo un error inesperado: {e}")
        return {"error": "Se produjo un error inesperado"}
    


actor = get_oscar_by_actor("brad pitt")
print(actor)

[(7445, 1995, 1996, 68, 'ACTOR IN A SUPPORTING ROLE', 'Brad Pitt', '12 Monkeys', False), (8909, 2008, 2009, 81, 'ACTOR IN A LEADING ROLE', 'Brad Pitt', 'The Curious Case of Benjamin Button', False), (9271, 2011, 2012, 84, 'ACTOR IN A LEADING ROLE', 'Brad Pitt', 'Moneyball', False), (9348, 2011, 2012, 84, 'BEST PICTURE', 'Michael De Luca, Rachael Horovitz and Brad Pitt, Producers', 'Moneyball', False), (9595, 2013, 2014, 86, 'BEST PICTURE', 'Brad Pitt, Dede Gardner, Jeremy Kleiner, Steve McQueen and Anthony Katagas, Producers', '12 Years a Slave', True), (9839, 2015, 2016, 88, 'BEST PICTURE', 'Brad Pitt, Dede Gardner and Jeremy Kleiner, Producers', 'The Big Short', False), (10277, 2019, 2020, 92, 'ACTOR IN A SUPPORTING ROLE', 'Brad Pitt', 'Once upon a Time...in Hollywood', True)]


In [6]:
from sqlalchemy import text

# Suponiendo que tienes 1000 registros por página
page_size = 100
page_number = 1  # La página que quieras cargar

query = text(f"SELECT * FROM oscar_awards LIMIT {page_size} OFFSET {(page_number - 1) * page_size}")

with engine.connect() as connection:
    result = connection.execute(query)
    for row in result:
        print(row)

(1, 1927, 1928, 1, 'ACTOR', 'Richard Barthelmess', 'The Noose', False)
(2, 1927, 1928, 1, 'ACTOR', 'Emil Jannings', 'The Last Command', True)
(3, 1927, 1928, 1, 'ACTRESS', 'Louise Dresser', 'A Ship Comes In', False)
(4, 1927, 1928, 1, 'ACTRESS', 'Janet Gaynor', '7th Heaven', True)
(5, 1927, 1928, 1, 'ACTRESS', 'Gloria Swanson', 'Sadie Thompson', False)
(6, 1927, 1928, 1, 'ART DIRECTION', 'Rochus Gliese', 'Sunrise', False)
(7, 1927, 1928, 1, 'ART DIRECTION', 'William Cameron Menzies', 'The Dove;', True)
(8, 1927, 1928, 1, 'ART DIRECTION', 'Harry Oliver', '7th Heaven', False)
(9, 1927, 1928, 1, 'CINEMATOGRAPHY', 'George Barnes', 'The Devil Dancer;', False)
(10, 1927, 1928, 1, 'CINEMATOGRAPHY', 'Charles Rosher', 'Sunrise', True)
(11, 1927, 1928, 1, 'CINEMATOGRAPHY', 'Karl Struss', 'Sunrise', True)
(12, 1927, 1928, 1, 'DIRECTING (Comedy Picture)', 'Lewis Milestone', 'Two Arabian Knights', True)
(13, 1927, 1928, 1, 'DIRECTING (Comedy Picture)', 'Ted Wilde', 'Speedy', False)
(14, 1927, 1928,